# **Moodle Logs Analysis**

This is a comprehensive analysis of Moodle logs from a training program at our institution. The dataset, which has been transformed to include essential metrics such as **activity_count**, **login_count**, **time**, **average_activity**, and **performance_bracket**, offers deep insights into learner engagement. In addition, a derived **at risk** column has been created based on heuristic rules to flag learners who might be struggling. 

The notebook is organized into multiple sections. Each section explains the analysis process and the insights gained from the data.


In [1]:
!pip install plotly

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler, LabelEncoder

sns.set(style='whitegrid', palette='muted', font_scale=1.1)
%matplotlib inline
print("Libraries imported successfully.")

Libraries imported successfully.


## **Data Loading**

The transformed data is loaded from the local CSV file **etl_output.csv**. This file is assumed to contain the merged and pre-processed log data from Moodle, incorporating key metrics that are essential for the analysis. Below, the first five rows of the dataset are printed to provide an initial look at the data.

In [3]:
try:
    df = pd.read_csv('../data/etl_output.csv')
    print("Transformed Data Sample:")
    print(df.head(5))
except Exception as e:
    print(f"Error loading transformed data: {e}")

Transformed Data Sample:
   user_id  activity_count  country gender  time  login_count  \
0      -10               1      NaN    NaN   NaN          NaN   
1       -1            2170      NaN    NaN   NaN          NaN   
2        0            7257      NaN    NaN   NaN          NaN   
3        1               3  Germany   Male   NaN          1.0   
4        2           45023  Germany   Male   NaN        169.0   

   average_activity performance_bracket  
0               NaN              top 1%  
1               NaN              others  
2               NaN              others  
3          3.000000              top 1%  
4        266.408284              others  


## 2. Data Overview

This section examines the structure and statistical summaries of the data. The DataFrame information, including data types and non-null counts, along with summary statistics, help in understanding the distribution and range of values. It also gives insight into the overall dimensions of the dataset, which is important for subsequent analysis and visualization.

In [4]:
print("DataFrame Info:")
df.info()

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1049 entries, 0 to 1048
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              1049 non-null   int64  
 1   activity_count       1049 non-null   int64  
 2   country              1046 non-null   object 
 3   gender               1046 non-null   object 
 4   time                 1040 non-null   float64
 5   login_count          600 non-null    float64
 6   average_activity     600 non-null    float64
 7   performance_bracket  1049 non-null   object 
dtypes: float64(3), int64(2), object(3)
memory usage: 65.7+ KB


In [5]:
print("\nSummary Statistics:")
display(df.describe(include='all'))



Summary Statistics:


,user_id,activity_count,country,gender,time,login_count,average_activity,performance_bracket
count,1049.000000,1049.000000,1046,1046,1040.000000,600.000000,600.000000,1049
unique,NaN,NaN,38,3,NaN,NaN,NaN,5
top,NaN,NaN,Ghana,Male,NaN,NaN,NaN,others
freq,NaN,NaN,319,700,NaN,NaN,NaN,734
mean,527.872259,398.049571,NaN,NaN,9291.388462,7.406667,77.331643,NaN
std,303.181563,1780.778421,NaN,NaN,38309.385128,14.444689,165.453963,NaN
min,-10.000000,1.000000,NaN,NaN,0.000000,1.000000,3.000000,NaN
25%,266.000000,12.000000,NaN,NaN,0.000000,1.000000,23.375000,NaN
50%,528.000000,41.000000,NaN,NaN,0.000000,2.000000,39.000000,NaN
75%,790.000000,124.000000,NaN,NaN,2618.750000,7.000000,71.181034,NaN


In [6]:
print("\nData Shape:", df.shape)


Data Shape: (1049, 8)


The examination of the DataFrame reveals that some columns (such as **country** and **gender**) contain missing values, which may require cleaning. The summary statistics indicate a heavy right skew in the **activity_count** variable and a high variance in **login_count**, both of which are important for understanding engagement differences among learners. Overall, the dataset's moderate size allows for detailed interactive analysis.

## **Exploratory Data Analysis (EDA)**

This section uses visualizations to explore the key metrics of the dataset. The objective is to reveal the underlying distributions, understand relationships between different engagement metrics, and identify potential outliers. Several types of plots are generated, each serving a specific purpose:

- **Histogram**: To show the frequency distribution of **activity_count**.
- **Bar Chart**: To illustrate the distribution of learners across different **performance_bracket** groups.
- **Scatter Plot**: To investigate the relationship between **login_count** and **average_activity**.
- **Box Plot**: To display the spread and outliers in **activity_count**.
- **Violin Plot**: To visualize the distribution and density of **login_count**.

### **Activity Counts**

In [7]:
fig_hist = px.histogram(
    df,
    x="activity_count",
    nbins=50,
    title="Distribution of Activity Counts",
    labels={"activity_count": "Activity Count"},
    template="plotly_white"
)
fig_hist.update_traces(marker_color="#86c5da")
fig_hist.update_layout(xaxis_title="Activity Count", yaxis_title="Frequency")
fig_hist.show()

The histogram clearly demonstrates a heavy right skew in **activity_count**, suggesting that while most learners have moderate activity, a small number show extremely high activity levels.

### **Performance Brackets**


In [8]:
perf_counts = df["performance_bracket"].value_counts().reset_index()
perf_counts.columns = ["performance_bracket", "num_learners"]

fig_bar = px.bar(
    perf_counts,
    x="performance_bracket",
    y="num_learners",
    title="Learners by Performance Bracket",
    labels={"performance_bracket": "Performance Bracket", "num_learners": "Number of Learners"},
    template="plotly_white",
    color="performance_bracket"
)
fig_bar.update_layout(xaxis={'categoryorder':'array', 'categoryarray':['top 1%','top 5%','top 10%','top 25%', 'others']})
fig_bar.show()

The bar chart shows that the majority of learners are classified in the "others" performance bracket, with fewer learners in higher performance groups. This suggests a generally moderate level of performance across the dataset.

### **Average Activity vs. Login Count**


In [9]:
fig_scatter = px.scatter(
    df,
    x="login_count",
    y="average_activity",
    color="performance_bracket",
    title="Average Activity per Login vs. Login Count",
    labels={"login_count": "Login Count", "average_activity": "Average Activity"},
    template="plotly_white"
)
fig_scatter.update_traces(marker=dict(size=8, line=dict(width=1, color='white')))
fig_scatter.show()

The scatter plot demonstrates a positive correlation between **login_count** and **average_activity**, although there is notable variability. This suggests that while increased logins are generally associated with higher activity per login, other factors and outliers may be influencing this relationship.

### **Activity Count**

In [10]:
fig_box = px.box(
    df,
    y="activity_count",
    points="all",
    title="Box Plot of Activity Count",
    labels={"activity_count": "Activity Count"},
    template="plotly_white"
)
fig_box.show()

### **Login Count**

In [11]:

fig_violin = px.violin(
    df,
    y="login_count",
    box=True,
    points="all",
    title="Violin Plot of Login Count",
    labels={"login_count": "Login Count"},
    template="plotly_white"
)
fig_violin.show()

Box and violin plots provide additional evidence of wide data dispersion and the presence of outliers in both **activity_count** and **login_count**.

##  **Outlier Analysis**

This section focuses on detecting outliers in **activity_count** using the Interquartile Range (IQR) method. Learners with **activity_count** values below or above the range [Q1 - 1.5×IQR, Q3 + 1.5×IQR] are flagged as outliers. Additionally, learners with values above the 95th percentile are considered high-activity and are analyzed further.

### **IQR-Based Outlier Detection**

In [12]:

Q1 = df['activity_count'].quantile(0.25)
Q3 = df['activity_count'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['activity_count'] < lower_bound) | (df['activity_count'] > upper_bound)]
print(f"Outliers in activity_count: {len(outliers)}")
display(outliers[['user_id', 'activity_count']].head(10))

Outliers in activity_count: 172


,user_id,activity_count
1,-1,2170
2,0,7257
4,2,45023
5,3,12922
7,5,1079
10,9,436
15,16,433
24,25,1894
35,39,500
36,40,350


The output quantifies the number of outliers, underscoring the variability in learner engagement and suggesting that some values may be extreme or anomalous.

###  **High-Activity Learners**

Learners with an **activity_count** above the 95th percentile are extracted for additional review. This group represents those learners who are exceptionally active and may either be highly engaged or indicate potential data anomalies.

In [13]:
high_threshold = df['activity_count'].quantile(0.95)
high_activity_learners = df[df['activity_count'] > high_threshold]
print(f"Number of high-activity learners: {len(high_activity_learners)}")
display(high_activity_learners[['user_id', 'activity_count']].head(10))

Number of high-activity learners: 53


,user_id,activity_count
1,-1,2170
2,0,7257
4,2,45023
5,3,12922
24,25,1894
38,42,4916
41,45,4241
44,48,2710
53,57,2836
54,58,2241


The analysis shows that many outliers exist, indicating large variability in engagement levels among learners. The high-activity subset, often with values above 40,000 in **activity_count**, requires careful interpretation to differentiate between genuinely high engagement and potential data errors.

## **At Risk Column Creation**

A new **at_risk** column is generated to flag learners who might be struggling. This heuristic flags learners whose **average_activity** is below the median while their **login_count** is above the median, suggesting that they log in frequently but do not engage deeply in each session. This dual condition helps identify learners who may need additional support.

In [14]:
threshold_avg_activity = df['average_activity'].median()
threshold_login = df['login_count'].median()

df['at_risk'] = np.where((df['average_activity'] < threshold_avg_activity) & (df['login_count'] > threshold_login), 1, 0)

print("At Risk Column Distribution:")
print(pd.Series(df['at_risk']).value_counts())

At Risk Column Distribution:
at_risk
0    918
1    131
Name: count, dtype: int64


## **Predictive Modeling**

Two predictive modeling approaches are implemented:

1. **Performance Bracket Classification**

    This model uses the observed **performance_bracket** (after label encoding) as the target variable. Engagement metrics (such as **activity_count**, **login_count**, **time**, and **average_activity**) are used as features to predict which performance group a learner belongs to.

2. **At Risk Classification**

    A second model is built using the newly created **at_risk** label as the target. This model aims to identify learners who, based on their engagement data, are likely to be struggling.

Both models are evaluated using standard metrics such as classification reports and confusion matrices, and further refined using hyperparameter tuning with GridSearchCV and cross-validation.

###  **Performance Bracket Classification**

The **performance_bracket** model first encodes the target variable using LabelEncoder. The data is then split into training and test sets before training a Random Forest Classifier.

In [15]:
features = ['activity_count', 'login_count', 'time', 'average_activity']
target = 'performance_bracket'

df_mod = df.dropna(subset=[target])
le = LabelEncoder()
df_mod[target] = le.fit_transform(df_mod[target])

print("Distribution of performance_bracket (encoded):")
print(pd.Series(df_mod[target]).value_counts())

Distribution of performance_bracket (encoded):
performance_bracket
0    734
3    164
2     96
1     28
4     27
Name: count, dtype: int64


Next, the data is split into training and testing sets, and a Random Forest Classifier is trained. The model's performance is evaluated using the classification report and confusion matrix.

In [16]:
X_train, X_test, y_train, y_test = train_test_split(df_mod[features].fillna(0), df_mod[target], test_size=0.3, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

print("Classification Report (Performance Bracket):")
print(classification_report(y_test, predictions))

print("Confusion Matrix (Performance Bracket):")
print(confusion_matrix(y_test, predictions))

Classification Report (Performance Bracket):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       222
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        29
           3       1.00      1.00      1.00        46
           4       1.00      1.00      1.00         8

    accuracy                           1.00       315
   macro avg       1.00      1.00      1.00       315
weighted avg       1.00      1.00      1.00       315

Confusion Matrix (Performance Bracket):
[[222   0   0   0   0]
 [  0  10   0   0   0]
 [  0   0  29   0   0]
 [  0   0   0  46   0]
 [  0   0   0   0   8]]


### **At Risk Classification**

In addition to the performance bracket model, a model is built to predict the newly created **at_risk** label. This model follows the same procedure, with the target variable now representing whether or not a learner is flagged as at risk.

In [17]:
target_at_risk = 'at_risk'

df_mod2 = df.dropna(subset=[target_at_risk])

X_train2, X_test2, y_train2, y_test2 = train_test_split(df_mod2[features].fillna(0), df_mod2[target_at_risk], test_size=0.3, random_state=42)

model_at_risk = RandomForestClassifier(n_estimators=100, random_state=42)
model_at_risk.fit(X_train2, y_train2)
predictions_at_risk = model_at_risk.predict(X_test2)

print("Classification Report (At Risk):")
print(classification_report(y_test2, predictions_at_risk))

print("Confusion Matrix (At Risk):")
print(confusion_matrix(y_test2, predictions_at_risk))

Classification Report (At Risk):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       272
           1       1.00      1.00      1.00        43

    accuracy                           1.00       315
   macro avg       1.00      1.00      1.00       315
weighted avg       1.00      1.00      1.00       315

Confusion Matrix (At Risk):
[[272   0]
 [  0  43]]


### **Model Refinement with Hyperparameter Tuning**

To improve model robustness, hyperparameter tuning is performed using GridSearchCV. This approach searches over a specified grid of parameters and uses cross-validation to find the best model settings for the Random Forest Classifier.

In [18]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5]
}

grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=3,
    scoring='accuracy',
    n_jobs=-1
)
grid_search.fit(X_train, y_train)

print("Best Parameters (Performance Bracket):", grid_search.best_params_)
print("Best Score (Performance Bracket):", grid_search.best_score_)

# Evaluate the best estimator on the test set
best_model = grid_search.best_estimator_
test_preds = best_model.predict(X_test)
print("\nClassification Report after Tuning (Performance Bracket):")
print(classification_report(y_test, test_preds))

# Perform cross-validation on the full dataset
scores = cross_val_score(best_model, df_mod[features].fillna(0), df_mod[target], cv=5)
print("Cross-validation scores (Performance Bracket):", scores)
print("Mean CV score (Performance Bracket):", scores.mean())

Best Parameters (Performance Bracket): {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 50}
Best Score (Performance Bracket): 1.0

Classification Report after Tuning (Performance Bracket):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       222
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        29
           3       1.00      1.00      1.00        46
           4       1.00      1.00      1.00         8

    accuracy                           1.00       315
   macro avg       1.00      1.00      1.00       315
weighted avg       1.00      1.00      1.00       315

Cross-validation scores (Performance Bracket): [1. 1. 1. 1. 1.]
Mean CV score (Performance Bracket): 1.0


The hyperparameter tuning results indicate that the model achieves high accuracy and generalizes well to unseen data. This suggests that engagement metrics are strong predictors of performance groups.

## **Advanced Analytics**

Additional techniques are applied to gain deeper insights into learner behavior:

###  **Clustering with K-Means**

Learners are grouped into distinct clusters using K-Means clustering. The engagement features are standardized prior to clustering. The cluster centroids (transformed back to the original scale) offer interpretable benchmarks for the average engagement levels within each group.

In [19]:
cluster_data = df[['activity_count', 'login_count', 'time']].fillna(0)
scaler = StandardScaler()
scaled_data = scaler.fit_transform(cluster_data)

kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(scaled_data)
df['cluster'] = kmeans.labels_

fig_cluster = px.scatter_3d(
    df,
    x='activity_count',
    y='login_count',
    z='time',
    color='cluster',
    title='3D Clustering of Learners',
    template='plotly_white'
)

fig_cluster.update_layout(scene_camera=dict(eye=dict(x=1.5, y=1.5, z=1), center=dict(x=0, y=0, z=0)))
fig_cluster.show()

centroids = scaler.inverse_transform(kmeans.cluster_centers_)
print("Cluster Centroids (original scale):")
print(centroids)

Cluster Centroids (original scale):
[[2.39443902e+02 2.93560976e+00 5.05914439e+03]
 [5.52613043e+03 5.50434783e+01 1.94670478e+05]
 [4.50230000e+04 1.69000000e+02 0.00000000e+00]]


### **Anomaly Detection with Local Outlier Factor (LOF)**

Local Outlier Factor (LOF) is applied to detect anomalies in the standardized engagement features. An interactive 3D scatter plot is created to visualize the anomalies with fixed camera settings, ensuring a stable view. This helps in identifying learners whose engagement patterns differ significantly from the norm.

In [20]:
lof = LocalOutlierFactor(n_neighbors=20)
lof_labels = lof.fit_predict(scaled_data)
df['lof_anomaly'] = (lof_labels == -1).astype(int)
print("Number of anomalies detected:", df['lof_anomaly'].sum())

anom_df = df[df['lof_anomaly'] == 1]
fig_anom = px.scatter_3d(
    anom_df,
    x='activity_count',
    y='login_count',
    z='time',
    color='lof_anomaly',
    title='3D Anomaly Detection (LOF)',
    template='plotly_white'
)

fig_anom.update_layout(scene_camera=dict(eye=dict(x=1.5, y=1.5, z=1), center=dict(x=0, y=0, z=0)))
fig_anom.show()

Number of anomalies detected: 113


##  **Insights**

The analysis reveals that the histogram of activity_count is heavily right-skewed, indicating that while the majority of learners exhibit moderate levels of activity, a small group is extraordinarily active. This skewness is significant because it implies that overall engagement might be disproportionately driven by a minority of learners, which could influence both overall performance and resource allocation.

In the scatter plot analysis, there is a noticeable positive trend between login_count and average_activity. Generally, as the number of logins increases, so does the average activity per login. However, this trend comes with significant variability, suggesting that other factors, including outliers, also significantly impact learner engagement. These discrepancies indicate that not all frequent logins result in high engagement, highlighting the importance of considering additional contextual factors.

The outlier analysis, conducted using the Interquartile Range (IQR) method, detected a large number of outliers. Particularly, high-activity learners—those whose activity counts exceed the 95th percentile—stand out remarkably. These extreme values can represent either students who are exceptionally engaged or potential data anomalies that warrant further investigation, as they may skew the overall analysis.

Predictive modeling further supports these findings. A Random Forest model built to predict the performance_bracket demonstrates strong accuracy when using engagement metrics as predictors. Additionally, a separate model created to predict the new at_risk column confirms that learners with low per-login engagement but high login counts can be effectively identified as potentially struggling. The use of hyperparameter tuning and cross-validation has enhanced the robustness of these models, confirming that the engagement metrics are reliable predictors for both performance grouping and risk assessment.

Advanced analytics methods add another layer of insight. Clustering with K-Means segments learners into distinct groups with interpretable centroids that reflect different engagement profiles. Anomaly detection using the Local Outlier Factor (LOF) identifies unusual engagement patterns that deviate from typical behavior. These analyses offer a deeper understanding of the varied learner profiles, which in turn provides a solid foundation for designing targeted interventions.

### **Conclusion**

The analysis confirms that the engagement metrics derived from Moodle logs provide valuable insights into learner performance. Both the observed performance groups and the at-risk indicator can be accurately predicted using these metrics, forming a strong foundation for data-driven interventions. Furthermore, advanced analytics reveal clear learner segments and anomalies, which support the development of real-time dashboards and proactive support systems. Future work should focus on further refining these predictive models, implementing continuous monitoring, and integrating these insights into operational systems to enhance overall learner outcomes.